In [4]:
import os

import fiona
import geopandas as gpd
import gpxpy
import numpy as np
import pandas as pd
from multiprocess.pool import Pool
from tqdm.notebook import tqdm, trange

fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'
storage = "/Volumes/easystore/Drones/"

calls_for_service = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"{storage}/calls-for-service-with-census-data.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)
flight_data = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/all-flights-manifest.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

In [8]:
anomolies = gpd.read_file(f"../../data/lingering-nd.geojson",chunksize=100000, dtype=str)

In [2]:
flight_data = flight_data[flight_data["department"] == "cvpd"].copy()
flight_data["match"] = flight_data["incident_id"].astype(str).apply(lambda x: x.strip().replace(' ','').replace('L','0').replace('l','0')[-5:])
flight_data["drone-date"] = pd.to_datetime(flight_data["date"]).dt.date

calls_for_service["match"] = (
    calls_for_service["Incident No."].astype(str).apply(lambda x: x.strip().replace(' ','').replace('l','0').replace('L','0')[-5:])
)
calls_for_service["cfs-date"] = pd.to_datetime(calls_for_service["Date"]).dt.date

In [3]:
matches = flight_data[flight_data["match"].isin(calls_for_service["match"])]

In [4]:
matches_joined_cfs = pd.merge(matches, calls_for_service, how="left", on="match")
matches_joined_cfs = matches_joined_cfs[
    [
        "time",
        "id",
        "date",
        "time_s",
        "address_map",
        "incident_id",
        "type",
        "Incident No.",
        "Date",
        "Description",
        "Priority",
        "Disposition",
        "Call Source",
        "Block Location",
        "Case No",
        "Address",
        "drone-date",
        "cfs-date"
    ]
]

In [5]:
matches_joined_cfs = matches_joined_cfs[
    matches_joined_cfs["drone-date"] == matches_joined_cfs["cfs-date"]
]
matches_joined_cfs.shape[0]

7688

In [6]:
not_matches = flight_data[~flight_data["match"].isin(calls_for_service["match"])]

In [7]:
flight_data[flight_data["incident_id"].isna()].columns

Index(['lat_map', 'lon_map', 'time', 'id', 'date', 'time_s', 'address_map',
       'incident_id', 'type', 'department', 'match', 'drone-date'],
      dtype='object')

In [8]:
not_matches[not_matches['incident_id'].astype(str) != "nan"]

,lat_map,lon_map,time,id,date,time_s,address_map,incident_id,type,department,match,drone-date
1,32.6501786,-117.0879108,2:43pm,c1795c8bb73272d54f70df7f7e228004,4-30-21,2:43pm,20 Fourth Ave,CVL035567,Subject down,cvpd,35567,2021-04-30
4,32.6274606,-117.0744953,11:30am,c347b051e39ae3a09b41234f9f254e8c,4-30-21,11:30am,700 Block of Third Ave,CVL035503,Domestic Violence,cvpd,35503,2021-04-30
9,32.6337859,-117.0831777,1:58am,42643525e446a922c7871f6470352652,4-30-21,1:58am,435 H St,CVL035549,Lude activity,cvpd,35549,2021-04-30
29,32.6461539,-117.0959105,11:42am,9bfbf4b4c1f89e02344a6420d846f1c0,4-28-21,11:42am,NaN,CVL034907,disturbance,cvpd,34907,2021-04-28
34,32.6408348,-117.0945315,5:10pm,39591c851c7c26c3182ffefa85076a78,4-27-21,5:10pm,"225 Broadway, 626 E St.","CVL34681, CVL34679","Suspicious Circumstances, Trespassing",cvpd,34679,2021-04-27
...,...,...,...,...,...,...,...,...,...,...,...,...
9602,32.6407168,-117.0751715,12:33pm,47c1b87a63f21a1ba783646dd0edb6bc,6-10-23,12:33pm,200 Cypress St.,CVL49085,Welfare Check,cvpd,49085,2023-06-10
9749,32.6333964,-117.0818848,12:53pm,bb1c49453ecd1b99570472ab37e7c06d,6-02-23,12:53pm,400 H St,CVL046589,Indecent exposure,cvpd,46589,2023-06-02
9750,32.6333964,-117.0818848,12:31pm,2025a1948770c18bf285e74f6de75950,6-02-23,12:31pm,400 H St,CVL046589,Indecent exposure,cvpd,46589,2023-06-02
9855,32.6061864,-117.054856,4:32pm,79518645967856af3e61f67a0868dc2f,7-27-23,4:32pm,1400 Tobias Dr,CV230,Robbery,cvpd,CV230,2023-07-27


In [80]:
calls_for_service['date'] = pd.to_datetime(calls_for_service['Date'])  
mask = (calls_for_service['date'] > '2022-04-01 18:00') & (calls_for_service['date'] < '2022-04-01 20:00')
calls_for_service.loc[mask].sort_values('date')

,Incident No.,Date,Description,Priority,Disposition,Call Source,Zip Code,Block Location,Case No,Address,...,Full FIPS (block),Full FIPS (tract),Metro/Micro Statistical Area Name,Metro/Micro Statistical Area Code,Metro/Micro Statistical Area Type,Combined Statistical Area Name,Combined Statistical Area Code,Metropolitan Division Area Name,Metropolitan Division Area Code,date
82356,220401L00026368,2022-04-01 18:04:15.913,Collision - No Details,2,REPORT TAKEN,citizen,91910,BROADWAY & J ST,NaN,"BROADWAY & J ST, Chula Vista, CA, 91910",...,060730130004008,06073013000,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-04-01 18:04:15.913
73030,220401L00026369,2022-04-01 18:06:54.103,Non-Injury Collision,2,ASSISTED,citizen,91910,400 BLOCK BROADWAY,2204369.0,"400 BLOCK BROADWAY, Chula Vista, CA, 91910",...,060730125012000,06073012501,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-04-01 18:06:54.103
121960,220401L00026371,2022-04-01 18:20:19.907,Trespassing,3,HANDLED NO REPORT,citizen,91910,200 BLOCK LANDIS AV,NaN,"200 BLOCK LANDIS AV, Chula Vista, CA, 91910",...,060730123021003,06073012302,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-04-01 18:20:19.907
59969,220401L00026372,2022-04-01 18:21:11.843,Duress Alarm,1,508 Left,citizen,91911,900 BLOCK BROADWAY,NaN,"900 BLOCK BROADWAY, Chula Vista, CA, 91911",...,060730131032000,06073013103,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-04-01 18:21:11.843
152031,220401L00026373,2022-04-01 18:30:20.820,Minor Injury Collision,2,ASSISTED,citizen,91911,OLYMPIC PW & OLEANDER AV,2204371.0,"OLYMPIC PW & OLEANDER AV, Chula Vista, CA, 91911",...,060730133094005,06073013309,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-04-01 18:30:20.820
158102,220401L00026374,2022-04-01 18:30:55.053,Drunk in Public,2,HANDLED NO REPORT,citizen,91911,MAIN ST & JACQUA ST,NaN,"MAIN ST & JACQUA ST, Chula Vista, CA, 91911",...,060730132051018,06073013205,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-04-01 18:30:55.053
59970,220401L00026376,2022-04-01 18:38:22.240,Shots Fired,2,HANDLED NO REPORT,officer,91911,900 BLOCK BROADWAY,NaN,"900 BLOCK BROADWAY, Chula Vista, CA, 91911",...,060730131032000,06073013103,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-04-01 18:38:22.240
71705,220401L00026377,2022-04-01 18:38:54.873,Duress Alarm,2,HANDLED NO REPORT,citizen,91911,900 BLOCK NACION AV,NaN,"900 BLOCK NACION AV, Chula Vista, CA, 91911",...,060730133011001,06073013301,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-04-01 18:38:54.873
165060,220401L00026378,2022-04-01 18:41:08.153,Other Incident,3,ASSISTED,citizen,91913,1400 BLOCK VIA HACIENDA,2204373.0,"1400 BLOCK VIA HACIENDA, Chula Vista, CA, 91913",...,060730134161006,06073013416,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-04-01 18:41:08.153
143867,220401L00026379,2022-04-01 18:44:45.870,Ensure Safety of Fire and Medical Personnel,2,ASSISTED,citizen,91911,100 BLOCK SIERRA WY,NaN,"100 BLOCK SIERRA WY, Chula Vista, CA, 91911",...,060730129002004,06073012900,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-04-01 18:44:45.870


In [85]:
flight_data[flight_data['id']=='fa84d14483eef42aab7e19f8b1d00551']

,lat_map,lon_map,time,id,date,time_s,address_map,incident_id,type,department
9052,32.6264829,-117.0325209,1:30pm,fa84d14483eef42aab7e19f8b1d00551,5-15-23,1:30pm,700 Telegraph Canyon Rd,2306551,Traffic Collision,cvpd


In [91]:
# L048211
calls_for_service[calls_for_service['Incident No.'].str.contains('6551')]
calls_for_service.sort_values('date').head()

,Incident No.,Date,Description,Priority,Disposition,Call Source,Zip Code,Block Location,Case No,Address,...,Full FIPS (block),Full FIPS (tract),Metro/Micro Statistical Area Name,Metro/Micro Statistical Area Code,Metro/Micro Statistical Area Type,Combined Statistical Area Name,Combined Statistical Area Code,Metropolitan Division Area Name,Metropolitan Division Area Code,date
0,210701L00054165,2021-07-01 00:17:06.233,Extra Patrol,5,HANDLED NO REPORT,officer,91910,300 BLOCK MOON VIEW DR,NaN,"300 BLOCK MOON VIEW DR, Chula Vista, CA, 91910",...,060730134121000,06073013412,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2021-07-01 00:17:06.233
16,210701L00054166,2021-07-01 00:20:07.827,Disturbance - General,2,HANDLED NO REPORT,citizen,91910,ELDER AV & KEARNEY ST,NaN,"ELDER AV & KEARNEY ST, Chula Vista, CA, 91910",...,060730130003006,06073013000,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2021-07-01 00:20:07.827
18,210701L00054167,2021-07-01 00:23:32.060,Disturbance - General,2,ASSISTED,citizen,91913,700 BLOCK EASTSHORE TE,2108248.0,"700 BLOCK EASTSHORE TE, Chula Vista, CA, 91913",...,060730134181003,06073013418,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2021-07-01 00:23:32.060
115,210701L00054168,2021-07-01 00:28:31.117,Disturbance - Party,3,HANDLED NO REPORT,citizen,91913,1600 BLOCK SANTA VENETIA ST,NaN,"1600 BLOCK SANTA VENETIA ST, Chula Vista, CA, ...",...,060730133171001,06073013317,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2021-07-01 00:28:31.117
627,210701L00054170,2021-07-01 00:40:26.430,Traffic Stop,5,HANDLED NO REPORT,officer,91910,E H ST & TIERRA DEL REY,NaN,"E H ST & TIERRA DEL REY, Chula Vista, CA, 91910",...,060730134093000,06073013409,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2021-07-01 00:40:26.430


In [49]:
flight_data['date'] = flight_data['date'].astype(str)
def compile_fl(df):


    df['fn'] = df['id'].apply(lambda x: f"/Volumes/easystore/Drones/flights/kml/{x}.kml")

    compiled_flights = []
    for _, row in df.iterrows():
        d = gpd.read_file(row['fn'], driver='KML')


        d['id'] = row['id']
        d['incident_type'] = row['type']
        d['incident_id'] = row['incident_id']
        d['address_map'] = row['address_map']
        d['date'] = row['date']
        d['time'] = row['time_s']

        compiled_flights.append(d)
    cf = pd.concat(compiled_flights,ignore_index=True)
    cf = gpd.GeoDataFrame(cf)
    cf['incident_type'] = cf['incident_type'].astype(str)
    return cf

In [50]:
c = compile_fl(flight_data[flight_data['date']=='4-01-22'])

/var/folders/nq/51nwndf52yqcj9x8h10zjpg80000gn/T/ipykernel_12148/1164878428.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fn'] = df['id'].apply(lambda x: f"/Volumes/easystore/Drones/flights/kml/{x}.kml")


In [52]:
c[['geometry','incident_type','id']].to_file("../data/7-11.geojson", driver='GeoJSON')

In [72]:
calls_[flight_data['date'].astype(str)=='4-01-22'].sort_values('date')

,lat_map,lon_map,time,id,date,time_s,address_map,incident_id,type,department
5091,32.6328996,-117.0906014,6:04pm,fa5419d7367804a0b54deb6c28074013,4-01-22,6:04pm,444 Broadway,L026369,Traffic Collision,cvpd
5116,32.639889,-117.0976342,9:17am,61d0fdc9b9488380aeec018f5c342053,4-01-22,9:17am,700 E,L026234,Robbery,cvpd
5115,32.639889,-117.0976342,9:34am,df7b691670880af84d6fd9a00d7fce98,4-01-22,9:34am,700 E,L026234,Robbery,cvpd
5114,32.6107542,-117.0476343,9:53am,8c7bd31d5c85bffb700ced5fdd8c9a16,4-01-22,9:53am,Max/E Prospect,L026242,Person Down,cvpd
5113,32.617434,-117.0716555,10:13am,4e990196b983b430e5f7e12e86634966,4-01-22,10:13am,330 Moss St,NaN,NaN,cvpd
5112,32.60310376234085,-117.05116125449219,10:25am,6ff107b5b4a9e3a131d25b961e19a4af,4-01-22,10:25am,SDG&E Park,L026254,Person Causing a Disturbance,cvpd
5111,32.6031467,-117.0879353,10:35am,a284f715ff126b5c1eb3231d8be7968c,4-01-22,10:35am,1225 Frontage,L026254,Fight,cvpd
5110,32.6099128,-117.0816837,11:16am,bb6854d1a120a15acd0c726d1d52a18b,4-01-22,11:16am,1100 Broadway,L026267,Traffic Collsion,cvpd
5109,32.63998774,-117.08273702,11:33am,7c0471c680ec028198fe78fec6d618fb,4-01-22,11:33am,400 C,L026271,Suicidal Person,cvpd
5108,32.6365157,-117.0959231,11:45am,f6851290905b0f4eee0263f4e01fc462,4-01-22,11:45am,670 F,L026274,Person Down,cvpd


In [ ]:
#sus flights
# 629a59ba8d4cac5b35edca159fd46ee5
#5c0e9e2d395f0cbae09c5ae183daddc7
#816f51b1ff04fa1b4a007bead6b20cc6
#729b90884e8e31ce418f68a1e01361dd